In [1]:
import pickle
import pandas as pd
import numpy as np
from tqdm import tqdm
import datetime
from geopy.distance import geodesic

Reading all the brands

In [2]:
brands_visits = pd.read_csv('data/revision_visits_revenue_2019.csv')
brands_visits['brand_standard'] = brands_visits['brand'].apply(lambda x: x.strip().lower()) # For comparison with catalog.tsv
brands_visits['date'] = brands_visits['date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d').date())
brands_visits = brands_visits.rename(columns={'brand': 'brand_visitation'})
brands_visits.head()

,date,PLACEKEY,visits_by_day,spend_by_day,brand_visitation,lat,lon,brand_standard
0,2019-06-01,zzw-222@62j-sgj-q2k,5,0.0,Orangetheory Fitness,42.350592,-71.153024,orangetheory fitness
1,2019-06-02,zzw-222@62j-sgj-q2k,1,0.0,Orangetheory Fitness,42.350592,-71.153024,orangetheory fitness
2,2019-06-03,zzw-222@62j-sgj-q2k,6,859.0,Orangetheory Fitness,42.350592,-71.153024,orangetheory fitness
3,2019-06-04,zzw-222@62j-sgj-q2k,6,30.0,Orangetheory Fitness,42.350592,-71.153024,orangetheory fitness
4,2019-06-05,zzw-222@62j-sgj-q2k,8,193.0,Orangetheory Fitness,42.350592,-71.153024,orangetheory fitness


Extracting all the unique brand placekeys

In [15]:
unique_brand_placekeys = brands_visits[['PLACEKEY', 'lat', 'lon']].groupby(['PLACEKEY'])[['lat', 'lon']].apply(lambda x: x.iloc[0]).reset_index()

In [16]:
# Function to calculate the distance between two coordinates
def calculate_distance_km(df_row):
    global lat_src
    global lon_src
    global placekey_src
    lat_dst = df_row['lat']
    lon_dst = df_row['lon']
    placekey_dst = df_row['PLACEKEY']
    
    dist_km = geodesic((lat_src, lon_src), (lat_dst, lon_dst)).km
    
    return pd.Series([placekey_src, placekey_dst, dist_km])

In [22]:
global lat_src
global lon_src
global placekey_src
neib_distance_km_df = pd.DataFrame(columns=['SRC_PLACEKEY', 'DST_PLACEKEY', 'Distance_Km'])

for i in tqdm(range(len(unique_brand_placekeys))):
    row_src = unique_brand_placekeys.iloc[i]
    placekey_src = row_src['PLACEKEY']
    lat_src = row_src['lat']
    lon_src = row_src['lon']

    temp_df = unique_brand_placekeys.apply(calculate_distance_km, axis=1).rename(columns={0:'SRC_PLACEKEY', 1: 'DST_PLACEKEY', 2: 'Distance_Km'})
        
    neib_distance_km_df = pd.concat([neib_distance_km_df, temp_df], axis=0).reset_index(drop=True)

  0%|          | 0/3 [00:00<?, ?it/s]/tmp/ipykernel_784200/974208352.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  neib_distance_km_df = pd.concat([neib_distance_km_df, temp_df], axis=0).reset_index(drop=True)
100%|██████████| 3/3 [00:06<00:00,  2.21s/it]


In [25]:
# with open('neib_distance_km.pkl', 'wb') as file:
#     pickle.dump(neib_distance_km_df, file)

### Calculating the brands for DST_PLACEKEY stores

In [7]:
brands_dict = brands_visits[['PLACEKEY', 'brand_visitation']].groupby(['PLACEKEY'])['brand_visitation'].apply(lambda x: x.iloc[0]).to_dict()

In [10]:
with open('neib_distance_km.pkl', 'rb') as file:
    neib_distance_km_df = pickle.load(file)

In [11]:
neib_distance_km_df.head()

,SRC_PLACEKEY,DST_PLACEKEY,Distance_Km
0,222-222@628-zxy-rc5,222-222@628-zxy-rc5,0.000000
1,222-222@628-zxy-rc5,222-222@628-zxy-tn5,0.255103
2,222-222@628-zxy-rc5,222-222@628-zy4-89z,3.765942
3,222-222@628-zxy-rc5,222-222@628-zz4-c5z,3.790174
4,222-222@628-zxy-rc5,222-222@628-zzp-f2k,5.166696


In [13]:
neib_distance_km_df['DST_BRAND'] = neib_distance_km_df['DST_PLACEKEY'].apply(lambda x: brands_dict[x])

In [15]:
with open('neib_distance_km_brand.pkl', 'wb') as file:
    pickle.dump(neib_distance_km_df, file)